# Context

When I tried to run a train where with the get_dataset tool from Darwin, I got an error message saying that some annotations contain negative values.
After contacting darwin for this issue, they said this behavior is expected. The workaround would be to trim the negative annotations.
So they sent me a python code called 00-trim_annotations_to_border_single_frame.py.

Steps:
- Download darwin annotations.
- Trim them.
- Upload and overwrite them into darwin.

In [1]:
import darwin

In [1]:
# Export the dataset from darwin with its current version 1, and split it into train, validation and test sets
!darwin dataset export digital-production/raven 3


/bin/bash: /home/naro/.conda/envs/raven/bin/darwin: /home/naro/Raven/.conda/bin/python: bad interpreter: No such file or directory


In [ ]:
!darwin dataset pull digital-production/raven
!darwin dataset split digital-production/raven --val-percentage 0.1 --test-percentage 0.2

In [ ]:
# Trim the annotations
!bash /home/naro/Raven/trim.sh

In [ ]:
# Get the list of new annotations (that are trimmed)
import os 

path = "/home/naro/Raven/annotations"
annotations_path = os.listdir(path)
annotations_path = [os.path.join(path, f) for f in annotations_path if f.endswith('json')] 
annotations_path[0:3] 

In [ ]:
from darwin.client import Client

API_KEY = "YyQSH5m.o7A4OmZfl4edpbfU6bom0THerkr08U9J"
client = Client.from_api_key(API_KEY)
dataset = client.get_remote_dataset("digital-production/raven")


In [ ]:
# Upload the annotations
import darwin.importer as importer
from darwin.client import Client
from darwin.importer import get_importer


parser = get_importer("darwin")
importer.import_annotations(dataset, parser, annotations_path, append=False)


In [ ]:
# Now export the new version again
!darwin dataset export digital-production/raven 2

In [ ]:
# Pull the version 2 of the dataset
!darwin dataset pull digital-production/raven:2

In [ ]:
# And split it again
!darwin dataset split digital-production/raven:2 --val-percentage 0.1 --test-percentage 0.2